## Agent Bricks - 情報抽出

[Agent Bricksの使用: 情報抽出](https://docs.databricks.com/aws/ja/generative-ai/agent-bricks/key-info-extraction)

## 1. bronze - PDFからテキスト抽出（OCR）

Volumeに置いたPDFからテキストを抽出します。<br>
抽出結果のテキストデータはテーブル`ab_receipt_bronze`として保存します。

- Agent Bricks -> 情報の抽出 -> PDF を使用


[ここに画像 or GIF 載せる]

## 2. silver - 情報の抽出エージェントを作成

テキストからJSON抽出するエージェントを作成します。<br>
`ab_receipt_bronze`テーブルの`text`カラム（pdfから抽出したテキスト）から、必要な情報をJSON形式で抽出するエージェントを作成します。<br>
Agent Bricksで作成したエージェントは、モデルサービングエンドポイントにモデルとして自動デプロイされます。<br>

- Agent Bricks -> 情報の抽出 -> ビルド

### 2-1. 抽出対象のデータ、抽出するJSON形式を定義する

[ここに画像 or GIF 載せる]

In [0]:
# print("""
# {
#   "発行日": "2024-12-21",
#   "請求先名": "ブリックスステム株式会社",
#   "請求元": {
#     "郵便番号": "150-0002",
#     "住所": "東京都渋谷区渋谷2-12-19 △△ヒル 1F",
#     "企業名": "株式会社パーパスメディア",
#     "部署名": "営業推進部"
#   },
#   "明細": [
#     {
#       "商品名": "○○○○○ サンプル タイプA",
#       "数量": 12345678,
#       "単価": 10,
#       "金額": 123456780
#     },
#     {
#       "商品名": "△△△ システム機器 ( 自動調整タイプ)",
#       "数量": 2,
#       "単価": 123456789,
#       "金額": 246913578
#     },
#     {
#       "商品名": "△△△ システムの取付作業",
#       "数量": 3,
#       "単価": 30000,
#       "金額": 90000
#     },
#     {
#       "商品名": "△△△ システムの操作説明 講習会",
#       "数量": 40,
#       "単価": 4000,
#       "金額": 160000
#     },
#     {
#       "商品名": "□□□□□□ 素材 ( ×× 在含む )",
#       "数量": 50,
#       "単価": 5000,
#       "金額": 250000
#     }
#   ],
#   "小計": 370870358,
#   "消費税": 37087035,
#   "合計金額": 407957393,
#   "備考": ""
# }
# """)

In [0]:
print("""
{
  "issue_date": "2024-12-21",
  "billing_name": "ブリックスステム株式会社",
  "supplier": {
    "zip_code": "104-0031",
    "address": "東京都中央区赤絪3丁目1-1 東京スケアガーデン14階",
    "company_name": "",
    "department_name": "営業部"
  },
  "details": [
    {
      "item_name": "○○○○○ サンプル タイプA",
      "item_quantity": 12345678,
      "item_unit_price": 10,
      "item_amount": 123456780
    },
    {
      "item_name": "△△△ システム機器 ( 自動調整タイプ)",
      "item_quantity": 2,
      "item_unit_price": 123456789,
      "item_amount": 246913578
    },
    {
      "item_name": "△△△ システムの取付作業",
      "item_quantity": 3,
      "item_unit_price": 30000,
      "item_amount": 90000
    },
    {
      "item_name": "△△△ システムの操作説明 講習会",
      "item_quantity": 40,
      "item_unit_price": 4000,
      "item_amount": 160000
    },
    {
      "item_name": "□□□□□□ 素材 ( ×× 在含む )",
      "item_quantity": 50,
      "item_unit_price": 5000,
      "item_amount": 250000
    }
  ],
  "subtotal": 370870358,
  "consumption_tax": 37087035,
  "total_amount": 407957393,
  "remarks": ""
}
""")

#### エージェント構成
##### ガイドライン

| カラム名 | データ型 | ガイドライン(JP) | ガイドライン(EN) |
|---|---|---|---|
| issue_date | string | 領収書/請求書の発行日（YYYY-MM-DD形式）。Null非許可。 | Date when the receipt or invoice was issued (YYYY-MM-DD, ISO 8601 format). Null not allowed. |
| billing_name | string | 請求先（個人または法人）。Null非許可。 | Name of the person or entity billed (individual or corporation). Null not allowed. |
| supplier | object | 請求元情報（会社・事業者に関するすべての情報の親オブジェクト）。Null非許可。 | Object containing all information about the issuer (company or business). Null allowed. |
| supplier › zip_code | string | 請求元住所の郵便番号（XXX-XXXX）。Null許可。 | Postal code of the issuer's address (e.g., XXX-XXXX in Japan). Null allowed. |
| supplier › address | string | 請求元の住所全文。都・市区町村・番地・建物名等含む。Null許可。 | Full address of the issuer, including prefecture, city, block number, and building name, etc. Null allowed. |
| supplier › company_name | string | 請求元の正式な会社名（法人格含む。「株式会社」など）。Null非許可。 | Official company name of the issuer, including corporation type (e.g., "Co., Ltd.", "Inc."). Null not allowed. |
| supplier › department_name | string | 請求元の部署名（担当部門）。Null許可。 | Department name of the issuer responsible for the transaction. Null allowed. |
| details | array[obj] | 商品・サービスごとの明細配列。複数アイテム情報を格納。Null許可。 | Array of product or service details; can include multiple items. Null not allowed. |
| details › items › item_name | string | 商品・サービスの名称。型番や仕様名記載も可。Null許可。 | Name of the product or service; model numbers or specifications may be included. Null allowed. |
| details › items › item_quantity | integer | 購入数量。カンマ区切りせず（区切り文字なしで）整数値で表記。Null許可。 | Quantity purchased. Enter as an integer, without any comma or delimiter. Null allowed. |
| details › items › item_unit_price | integer | 商品単位あたりの単価。日本円の整数値でカンマ区切りせず（区切り文字なしで）表記。Null許可。 | Unit price per item, as an integer without any comma or separator, in local currency. No decimals. Null allowed. |
| details › items › item_amount | integer | 明細ごとの合計金額（数量×単価）。日本円の整数値でカンマ区切りせず（区切り文字なしで）表記。Null許可。| Line item total (quantity × unit price), entered as an integer, without commas or separators. Local currency. Null allowed. |
| subtotal | integer | 全明細の合計金額（税抜）。日本円の整数値でカンマ区切りせず（区切り文字なしで）表記。Null許可。| Total of all line items before tax; enter as integer with no commas or separators. Local currency. Null allowed. |
| consumption_tax | integer | 小計に適用する消費税額。日本円の整数値でカンマ区切りせず（区切り文字なしで）表記。Null非許可。 | Consumption tax applied to subtotal (calculated by tax rate). Specify as integer, no commas or delimiters. Null not allowed. |
| total_amount | integer | 支払総額（小計＋消費税）。日本円の整数値でカンマ区切りせず（区切り文字なしで）表記。Null非許可。 | Total payment amount (subtotal plus tax). Enter as integer, without any thousands separators, in local currency. Null not allowed. |
| remarks | string | 注意事項・特記事項。Null許可。 | Additional notes or remarks. This field can be left blank if not needed. Null allowed. |

##### 指示
```
```

In [0]:
print("""
{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "title": "Generated Schema",
  "type": "object",
  "properties": {
    "issue_date": {
      "description": "領収書/請求書の発行日（YYYY-MM-DD形式）。Null非許可。",
      "anyOf": [
        {
          "type": "string"
        },
        {
          "type": "null"
        }
      ]
    },
    "billing_name": {
      "description": "請求先（個人または法人）。Null非許可。",
      "anyOf": [
        {
          "type": "string"
        },
        {
          "type": "null"
        }
      ]
    },
    "supplier": {
      "description": "請求元情報（会社・事業者に関するすべての情報の親オブジェクト）。Null非許可。",
      "anyOf": [
        {
          "type": "object",
          "properties": {
            "zip_code": {
              "anyOf": [
                {
                  "type": "string"
                },
                {
                  "type": "null"
                }
              ],
              "description": "請求元住所の郵便番号（XXX-XXXX）。Null許可。"
            },
            "address": {
              "anyOf": [
                {
                  "type": "string"
                },
                {
                  "type": "null"
                }
              ],
              "description": "請求元の住所全文。都・市区町村・番地・建物名等含む。Null許可。"
            },
            "company_name": {
              "anyOf": [
                {
                  "type": "string"
                },
                {
                  "type": "null"
                }
              ],
              "description": "請求元の正式な会社名（法人格含む。「株式会社」など）。Null非許可。"
            },
            "department_name": {
              "anyOf": [
                {
                  "type": "string"
                },
                {
                  "type": "null"
                }
              ],
              "description": "請求元の部署名（担当部門）。Null許可。"
            }
          },
          "required": [
            "zip_code",
            "address",
            "company_name",
            "department_name"
          ]
        },
        {
          "type": "null"
        }
      ]
    },
    "details": {
      "description": "商品・サービスごとの明細配列。複数アイテム情報を格納。Null許可。",
      "anyOf": [
        {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "item_name": {
                "anyOf": [
                  {
                    "type": "string"
                  },
                  {
                    "type": "null"
                  }
                ],
                "description": "商品・サービスの名称。型番や仕様名記載も可。Null許可。"
              },
              "item_quantity": {
                "anyOf": [
                  {
                    "type": "integer"
                  },
                  {
                    "type": "null"
                  }
                ],
                "description": "購入数量。カンマ区切りせず（区切り文字なしで）整数値で表記。Null許可。"
              },
              "item_unit_price": {
                "anyOf": [
                  {
                    "type": "integer"
                  },
                  {
                    "type": "null"
                  }
                ],
                "description": "商品単位あたりの単価。日本円の整数値でカンマ区切りせず（区切り文字なしで）表記。Null許可。"
              },
              "item_amount": {
                "anyOf": [
                  {
                    "type": "integer"
                  },
                  {
                    "type": "null"
                  }
                ],
                "description": "明細ごとの合計金額（数量×単価）。日本円の整数値でカンマ区切りせず（区切り文字なしで）表記。Null許可。"
              }
            },
            "required": [
              "item_name",
              "item_quantity",
              "item_unit_price",
              "item_amount"
            ]
          }
        },
        {
          "type": "null"
        }
      ]
    },
    "subtotal": {
      "description": "全明細の合計金額（税抜）。日本円の整数値でカンマ区切りせず（区切り文字なしで）表記。Null許可。",
      "anyOf": [
        {
          "type": "integer"
        },
        {
          "type": "null"
        }
      ]
    },
    "consumption_tax": {
      "description": "小計に適用する消費税額。日本円の整数値でカンマ区切りせず（区切り文字なしで）表記。Null非許可。",
      "anyOf": [
        {
          "type": "integer"
        },
        {
          "type": "null"
        }
      ]
    },
    "total_amount": {
      "description": "支払総額（小計＋消費税）。日本円の整数値でカンマ区切りせず（区切り文字なしで）表記。Null非許可。",
      "anyOf": [
        {
          "type": "integer"
        },
        {
          "type": "null"
        }
      ]
    },
    "remarks": {
      "description": "注意事項・特記事項。Null許可。",
      "anyOf": [
        {
          "type": "string"
        },
        {
          "type": "null"
        }
      ]
    }
  },
  "required": [
    "issue_date",
    "billing_name",
    "supplier",
    "details",
    "subtotal",
    "consumption_tax",
    "total_amount",
    "remarks"
  ]
}
""")

### 2-2. ガイドラインを定義する

## 3. silver - SQLを通してJSON抽出エージェントを使いテーブル保存

2でモデルサービングエンドポイントにデプロイされたモデルは、REST APIとして外部アプリからアクセスして活用できます。<br>
Databricks内であれば、Databricks SQL の AI関数 `ai_query()` を通して活用できます。<br>

ここでは、AI関数 `ai_query()` を活用し、SQLでエージェントを呼び出し、<br>
テキスト情報をJSONに変換 & テーブル`ab_receipt_silver`を作成します。<br>

- 作業場所: Notebook or SQLエディタ
- モデルサービングエンドポイント名: 例）`kie-000000xx-endpoint` ※Agent Bricksで作成した実際のエンドポイント名を使ってください

In [0]:
%run ./00_config

In [0]:
ENDPOINT_NAME = "kie-dae38944-endpoint"

In [0]:
# テキスト情報から必要項目をJSON形式で切り出して、Silverをテーブルを作成
spark.sql(f"""
CREATE OR REPLACE TABLE {MY_CATALOG}.{MY_SCHEMA}.ab_receipt_silver AS
WITH query_results AS (
  SELECT `text` AS input,
    ai_query(
      '{ENDPOINT_NAME}',
      input,
      failOnError => false
    ) AS response
  FROM (
    SELECT text
    FROM {MY_CATALOG}.{MY_SCHEMA}.ab_receipt_bronze
  )
)
SELECT
  input,
  response.result AS response,
  response.errorMessage AS error
FROM query_results
""")

In [0]:
# テキスト情報から必要項目をJSON形式で切り出して、Silverをテーブルを作成
print(f"""
CREATE OR REPLACE TABLE {MY_CATALOG}.{MY_SCHEMA}.ab_receipt_silver AS
WITH query_results AS (
  SELECT `text` AS input,
    ai_query(
      '{ENDPOINT_NAME}',
      input,
      failOnError => false
    ) AS response
  FROM (
    SELECT text
    FROM {MY_CATALOG}.{MY_SCHEMA}.ab_receipt_bronze
  )
)
SELECT
  input,
  response.result AS response,
  response.errorMessage AS error
FROM query_results
""")

In [0]:
display(
  spark.sql(f"select * from {MY_CATALOG}.{MY_SCHEMA}.ab_receipt_silver")
)

## 4. gold - JSON（半構造化）をクエリで構造化テーブルに展開

先ほど保存した`receipt_silver`テーブルをもとに、SQLを用いてJSONデータを分析しやすい構造化テーブルに展開します。

In [0]:
df = spark.sql(f"""
-- VARIANT型response列をto_jsonで一度STRING型に変換し、from_jsonでパース
CREATE OR REPLACE TABLE {MY_CATALOG}.{MY_SCHEMA}.ab_receipt_gold AS
SELECT
  uuid() AS id,
  parsed.issue_date AS issue_date,                              -- 発行日
  parsed.billing_name AS billing_name,                          -- 請求先名
  parsed.supplier.zip_code AS supplier_zip,                     -- 請求元.郵便番号
  parsed.supplier.address AS supplier_address,                  -- 請求元.住所
  parsed.supplier.company_name AS supplier_company,             -- 請求元.企業名
  parsed.supplier.department_name AS supplier_department,       -- 請求元.部署名
  detail.item_name AS item_name,                                -- 商品名
  CAST(detail.item_quantity AS BIGINT) AS item_qty,             -- 詳細.数量
  CAST(detail.item_unit_price AS BIGINT) AS item_unit_price,    -- 詳細.単価
  CAST(detail.item_amount AS BIGINT) AS item_amount,            -- 詳細.金額
  CAST(parsed.subtotal AS BIGINT) AS subtotal,                  -- 小計（税抜）
  CAST(parsed.consumption_tax AS BIGINT) AS consumption_tax,    -- 消費税金額
  CAST(parsed.total_amount AS BIGINT) AS total_amount_with_tax, -- 合計金額（税込）
  parsed.remarks AS remarks                                     -- 備考
FROM (
  SELECT
    *,
    from_json(
      to_json(response),
      'STRUCT<
         issue_date:STRING,
         billing_name:STRING,
         supplier:STRUCT<
           zip_code:STRING,
           address:STRING,
           company_name:STRING,
           department_name:STRING
         >,
         details:ARRAY<
           STRUCT<
             item_name:STRING,
             item_quantity:DOUBLE,
             item_unit_price:DOUBLE,
             item_amount:DOUBLE
           >
         >,
         subtotal:DOUBLE,
         consumption_tax:DOUBLE,
         total_amount:DOUBLE,
         remarks:STRING
       >'
    ) AS parsed
  FROM {MY_CATALOG}.{MY_SCHEMA}.ab_receipt_silver
)
LATERAL VIEW
  EXPLODE(parsed.details) details AS detail
;
""")

In [0]:
print(f"""
-- VARIANT型response列をto_jsonで一度STRING型に変換し、from_jsonでパース
CREATE OR REPLACE TABLE {MY_CATALOG}.{MY_SCHEMA}.ab_receipt_gold AS
SELECT
  uuid() AS id,
  parsed.issue_date AS issue_date,                              -- 発行日
  parsed.billing_name AS billing_name,                          -- 請求先名
  parsed.supplier.zip_code AS supplier_zip,                     -- 請求元.郵便番号
  parsed.supplier.address AS supplier_address,                  -- 請求元.住所
  parsed.supplier.company_name AS supplier_company,             -- 請求元.企業名
  parsed.supplier.department_name AS supplier_department,       -- 請求元.部署名
  detail.item_name AS item_name,                                -- 商品名
  CAST(detail.item_quantity AS BIGINT) AS item_qty,             -- 詳細.数量
  CAST(detail.item_unit_price AS BIGINT) AS item_unit_price,    -- 詳細.単価
  CAST(detail.item_amount AS BIGINT) AS item_amount,            -- 詳細.金額
  CAST(parsed.subtotal AS BIGINT) AS subtotal,                  -- 小計（税抜）
  CAST(parsed.consumption_tax AS BIGINT) AS consumption_tax,    -- 消費税金額
  CAST(parsed.total_amount AS BIGINT) AS total_amount_with_tax, -- 合計金額（税込）
  parsed.remarks AS remarks                                     -- 備考
FROM (
  SELECT
    *,
    from_json(
      to_json(response),
      'STRUCT<
         issue_date:STRING,
         billing_name:STRING,
         supplier:STRUCT<
           zip_code:STRING,
           address:STRING,
           company_name:STRING,
           department_name:STRING
         >,
         details:ARRAY<
           STRUCT<
             item_name:STRING,
             item_quantity:DOUBLE,
             item_unit_price:DOUBLE,
             item_amount:DOUBLE
           >
         >,
         subtotal:DOUBLE,
         consumption_tax:DOUBLE,
         total_amount:DOUBLE,
         remarks:STRING
       >'
    ) AS parsed
  FROM {MY_CATALOG}.{MY_SCHEMA}.ab_receipt_silver
)
LATERAL VIEW
  EXPLODE(parsed.details) details AS detail
;
""")

In [0]:
display(
  spark.sql(f"select * from {MY_CATALOG}.{MY_SCHEMA}.ab_receipt_gold")
)

In [0]:
# 変数定義
TABLE_PATH = f'{MY_CATALOG}.{MY_SCHEMA}.ab_receipt_gold'                 # テーブルパス
PK_CONSTRAINT_NAME = f'pk_ab_receipt_gold'                               # 主キー

# NOT NULL制約の追加
columns_to_set_not_null = [
    'id']

for column in columns_to_set_not_null:
    spark.sql(f"""
    ALTER TABLE {TABLE_PATH}
    ALTER COLUMN {column} SET NOT NULL;
    """)

# 主キー設定
spark.sql(f'''
ALTER TABLE {TABLE_PATH} DROP CONSTRAINT IF EXISTS {PK_CONSTRAINT_NAME};
''')

spark.sql(f'''
ALTER TABLE {TABLE_PATH}
ADD CONSTRAINT {PK_CONSTRAINT_NAME} PRIMARY KEY (id);
''')

# # チェック
# display(
#     spark.sql(f'''
#     DESCRIBE EXTENDED {TABLE_PATH}
#     '''))

In [0]:
certified_tag = 'system.Certified'

try:
    spark.sql(f"ALTER TABLE ab_receipt_gold SET TAGS ('{certified_tag}')")
    print(f"認定済みタグ '{certified_tag}' の追加が完了しました。")

except Exception as e:
    print(f"認定済みタグ '{certified_tag}' の追加中にエラーが発生しました: {str(e)}")
    print("このエラーはタグ機能に対応していないワークスペースで実行した場合に発生する可能性があります。")

In [0]:
# テーブル名
table_name = f'{MY_CATALOG}.{MY_SCHEMA}.ab_receipt_gold'

# テーブルコメント
comment = """
テーブル名：`ab_receipt_gold / 領収書（ゴールド）`
説明：領収書データをパースして構造化したテーブルです。分析用に使います。
"""
spark.sql(f'COMMENT ON TABLE {table_name} IS "{comment}"')

# カラムコメント
column_comments = {
    "id": "自動採番したユニークID",
    "issue_date": "発行日",
    "billing_name": "請求先名",
    "supplier_zip": "請求元 郵便番号",
    "supplier_address": "請求元 住所",
    "supplier_company": "請求元 企業名",
    "supplier_department": "請求元 部署名",
    "item_name": "明細 商品名",
    "item_qty": "明細 数量",
    "item_unit_price": "明細 単価",
    "item_amount": "明細 金額",
    "subtotal": "小計（税抜）",
    "consumption_tax": "消費税金額",
    "total_amount_with_tax": "合計金額（税込）",
    "remarks": "備考"
}

for column, comment in column_comments.items():
    escaped_comment = comment.replace("'", "\\'")
    sql_query = f"ALTER TABLE {table_name} ALTER COLUMN {column} COMMENT '{escaped_comment}'"
    spark.sql(sql_query)